In [79]:
import igl
import polyscope as ps
import numpy as np 

import os 
import sys

# Add hananLab to path
#path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))
path = os.path.dirname(os.getcwd())
sys.path.append(path)

from hanan.geometry.utils import read_obj, unit, vec_dot

In [65]:
# Vertices of a mesh with 4 triangles
v = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0.5]], dtype=np.float64)

# Faces of a mesh with 4 triangles
f = np.array([[0, 1, 2], [1, 2, 3], [0, 1, 3]])

# Points to be queried
p = np.array([[-0.1, -1.2, 0.3]])

# Query the closest point on the mesh
dist, fid, cp = igl.point_mesh_squared_distance(p, v, f)

print("dist: ", dist)
print("Face id: ", fid)
print("Closest points: ", cp)
print("point: ", p)

ps.init()
ps.remove_all_structures()

ps.register_surface_mesh("mesh", v, f)
ps.register_point_cloud("point", p)
ps.register_point_cloud("c point", np.array([cp]))


ps.show()

dist:  1.45
Face id:  2
Closest points:  [0.  0.  0.3]
point:  [[-0.1 -1.2  0.3]]


In [133]:
# Read start mesh
sv, sf = igl.read_triangle_mesh("/home/anthony/GeometricProcessing/Quad_remesh/source/test_remeshed_1_start.obj")
# Read deformed mesh
dv, df = igl.read_triangle_mesh("/home/anthony/GeometricProcessing/Quad_remesh/source/test_remeshed_1_deformed.obj")
# Read Quad mesh
qv, qf = read_obj("/home/anthony/GeometricProcessing/Quad_remesh/source/test_remeshed_1_quad.obj")


# Query the closest point on the mesh
dist, f_id, qv_p = igl.point_mesh_squared_distance(qv, dv, df)

# Compute barycentric coordinates in the deformed mesh
bc = igl.barycentric_coordinates_tri(qv_p, dv[df[f_id, 0]], dv[df[f_id, 1]], dv[df[f_id, 2]])

# Remap the barycentric coordinates to the start mesh
sv_p = bc[:, 0][:, None] * sv[df[f_id, 0]] +  bc[:, 1][:, None]* sv[df[f_id, 1]] +  bc[:, 2][:, None] * sv[df[f_id, 2]]

  
# Measure planarity of quads
qf2 = np.zeros((len(qf), 4), dtype=np.int64)
for i, f in enumerate(qf):
    try:
        qf2[i] = np.array([f[0], f[1], f[2], f[3]])
    except:
        print("Error: ", f)


p0, p1, p2, p3 = sv_p[qf2[:, 0]], sv_p[qf2[:, 1]], sv_p[qf2[:, 2]], sv_p[qf2[:, 3]]

# Compute the normal of the quad
n = unit(np.cross(p2 - p0, p3 - p1))

# edges 
e0 = unit(p1 - p0)
e1 = unit(p2 - p1)
e2 = unit(p3 - p2)
e3 = unit(p0 - p3)

# Compute the angle between the normal and the edges
planarity =(vec_dot(n, e0) + vec_dot(n, e1) + vec_dot(n, e2) + vec_dot(n, e3)) 

# Compute sphere centers
sph_c, sph_r= compute_centers_spheres(sv_p, qf2)

print(sph_c)
ps.init()
ps.remove_all_structures()


ps.register_point_cloud("centers spheres", sph_c)
ps.register_surface_mesh("quad", qv, qf)
qrmp = ps.register_surface_mesh("quad remaped", sv_p, qf)
qrmp.add_scalar_quantity("planarity", planarity, defined_on="faces")




ps.show()



  o test_remeshed_1_start
  o test_remeshed_1_deformed
/home/anthony/Hanan Lab/hananLab/hanan/geometry/utils.py:10: RuntimeWarning: invalid value encountered in divide
  unit_v = v/np.linalg.norm(v, axis=1)[:, None]


Error:  [1037, 73, 72]
[[ -5.5954996   -7.25612237 -11.7313217 ]
 [ 11.23728727  19.67051705  32.00979807]
 [ -2.20523062  -2.13340148  -3.28780564]
 ...
 [  0.43565781  -0.83140705  -1.27104361]
 [  1.10218956  -0.91936384  -2.52764619]
 [ -1.55679496   4.31943945   7.37918703]]


/home/anthony/Hanan Lab/hananLab/hanan/geometry/utils.py:8: RuntimeWarning: invalid value encountered in divide
  unit_v = v/np.linalg.norm(v)


In [132]:
def fit_sphere(p0, p1, p2, p3):
    p0_2 = p0@p0
    p1_2 = p1@p1
    p2_2 = p2@p2
    p3_2 = p3@p3

    M  = np.array([
        [p0_2, p0[0], p0[1], p0[2], 1],
        [p1_2, p1[0], p1[1], p1[2], 1],
        [p2_2, p2[0], p2[1], p2[2], 1],
        [p3_2, p3[0], p3[1], p3[2], 1],
                ])
    
    A  =  np.linalg.det(M[:, 1:])
    Bx = -np.linalg.det(M[:, [0, 2, 3, 4]])
    By =  np.linalg.det(M[:, [0, 1, 3, 4]])
    Bz = -np.linalg.det(M[:, [0, 1, 2, 4]])
    B  = - np.array([Bx, By, Bz]) 
    C  =  np.linalg.det(M[:, :-1])

    if abs(A) < 1e-8:
        # Not a sphere, then return a plane at the center of the quad
        br = (p0 + p1 + p2 + p3)/4

        # Normal 
        n = unit(B)

        return br, n, "plane"
    else:
        center = B / (2 * A)
        radius = np.sqrt( (B@B - 4 * A * C)/ (4 * A**2))

        return center, radius, "sphere"
    
def compute_centers_spheres(qv, qf):
    """
        Function to compute the center of the sphere that fits each quad
    """

    centers = np.zeros((len(qf), 3), dtype=np.float64)
    radius = np.zeros((len(qf), 1), dtype=np.float64)
    
    for i, f in enumerate(qf):
        v0, v1, v2, v3 = qv[f]

        center, rad, tp = fit_sphere(v0, v1, v2, v3)

        if tp == "sphere":
            centers[i] = center
            radius[i] = rad
        
    return centers, radius



In [109]:


    

# Compute the center and radius of the sphere
p0 = np.array([0, 0, 0])
p1 = np.array([1, 0, 0])
p2 = np.array([0, 1, 0])
p3 = np.array([0, 0, 1])

c, r = fit_sphere(p0, p1, p2, p3)

print("center: ", c)
print("radius: ", r)
print("distance p0:", np.linalg.norm(c - p0)) 
print("distance p1:", np.linalg.norm(c - p1))
print("distance p2:", np.linalg.norm(c - p2))
print("distance p3:", np.linalg.norm(c - p3))


center:  [0.5 0.5 0.5]
radius:  0.8660254037844386
distance p0: 0.8660254037844386
distance p1: 0.8660254037844386
distance p2: 0.8660254037844386
distance p3: 0.8660254037844386
